In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,cross_val_score
from numpy import mean,std
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,precision_score,recall_score
import warnings 
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [44]:
data = pd.read_csv('../Datasets/oversampled_data.csv')
del data['Unnamed: 0']
data.head()

,account_length,voice_mail_plan,voice_mail_messages,day_mins,evening_mins,night_mins,international_mins,customer_service_calls,international_plan,day_calls,day_charge,evening_calls,evening_charge,night_calls,night_charge,international_calls,international_charge,total_charge,churn
0,128,1,25,265.1,197.4,244.7,10.0,1,0,110,45.07,99,16.78,91,11.01,3,2.70,75.56,0
1,107,1,26,161.6,195.5,254.4,13.7,1,0,123,27.47,103,16.62,103,11.45,3,3.70,59.24,0
2,137,0,0,243.4,121.2,162.6,12.2,0,0,114,41.38,110,10.30,104,7.32,5,3.29,62.29,0
3,84,0,0,299.4,61.9,196.9,6.6,2,1,71,50.90,88,5.26,89,8.86,7,1.78,66.80,0
4,75,0,0,166.7,148.3,186.9,10.1,3,1,113,28.34,122,12.61,121,8.41,3,2.73,52.09,0


### Scaling an dseperating X and y

In [45]:
std_scr = StandardScaler()
X = data.drop(labels='churn',axis=1)
cols = X.columns
X = pd.DataFrame(std_scr.fit_transform(X))
X.columns = cols
y = data[['churn']]
X.shape,y.shape

((5700, 18), (5700, 1))

### Train Test Split

In [46]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=12,stratify=y)

### Dict to store performance metrics

In [47]:
my_dict = {'model':[],'train accuracy':[], 'test accuracy': [],'precision':[],'recall':[]}

### Method to predit and calculate model performance

In [48]:
def populate_matrix(fittedModel):
    y_pred_train = fittedModel.predict(x_train)
    train_accuracy = round(accuracy_score(y_train,y_pred_train)*100,2)
    
    y_pred = fittedModel.predict(x_test)
    test_accuracy = round(accuracy_score(y_test,y_pred)*100,2)
    precision = round(precision_score(y_test,y_pred)*100,2)
    recall = round(recall_score(y_test,y_pred)*100,2)
    model = str(fittedModel)
    my_dict["model"].append(model)
    my_dict["train accuracy"].append(train_accuracy)
    my_dict["test accuracy"].append(test_accuracy)
    my_dict["precision"].append(precision)
    my_dict["recall"].append(recall)


In [49]:
def cross_validate_model(model):
    # evaluate model
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    
    n_scores = cross_val_score(estimator=model,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


## Logistic Regression

In [50]:
# evaluate model
logistic = LogisticRegression()

cross_validate_model(logistic)


Accuracy: 0.776 (0.016)


### Model Traning

In [51]:
logistic.fit(x_train,y_train)
populate_matrix(logistic)
print(my_dict)

{'model': ['LogisticRegression()'], 'train accuracy': [78.31], 'test accuracy': [76.75], 'precision': [75.63], 'recall': [78.95]}


## DecisionTree

In [52]:
dtree = DecisionTreeClassifier()
# evaluate model
cross_validate_model(dtree)

Accuracy: 0.983 (0.005)


### Model Training

In [53]:
dtree.fit(x_train,y_train)
populate_matrix(dtree)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()'], 'train accuracy': [78.31, 100.0], 'test accuracy': [76.75, 98.25], 'precision': [75.63, 96.77], 'recall': [78.95, 99.82]}


## Random Forest

In [54]:
rf = RandomForestClassifier()
# evaluate model
cross_validate_model(rf)

Accuracy: 0.999 (0.001)


### Model Traning

In [55]:
rf.fit(x_train,y_train)
populate_matrix(rf)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()'], 'train accuracy': [78.31, 100.0, 100.0], 'test accuracy': [76.75, 98.25, 99.91], 'precision': [75.63, 96.77, 100.0], 'recall': [78.95, 99.82, 99.82]}


## Gradient Boosting

In [56]:
gboost = GradientBoostingClassifier()
# evaluate model
cross_validate_model(gboost)

Accuracy: 0.940 (0.008)


### Model Training

In [57]:
gboost.fit(x_train,y_train)
populate_matrix(gboost)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()'], 'train accuracy': [78.31, 100.0, 100.0, 94.61], 'test accuracy': [76.75, 98.25, 99.91, 94.12], 'precision': [75.63, 96.77, 100.0, 99.8], 'recall': [78.95, 99.82, 99.82, 88.42]}


## SVM

In [58]:
svm = SVC()
# evaluate model
cross_validate_model(svm)

Accuracy: 0.911 (0.011)


### Model Training

In [59]:
svm.fit(x_train,y_train)
populate_matrix(svm)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()'], 'train accuracy': [78.31, 100.0, 100.0, 94.61, 92.59], 'test accuracy': [76.75, 98.25, 99.91, 94.12, 89.3], 'precision': [75.63, 96.77, 100.0, 99.8, 89.44], 'recall': [78.95, 99.82, 99.82, 88.42, 89.12]}


## ANN

In [60]:
ann_model = Sequential()
ann_model.add(Dense(20,input_dim=x_train.shape[1],activation='relu'))
ann_model.add(Dense(10,activation='relu'))
ann_model.add(Dense(5,activation='relu'))
ann_model.add(Dense(1,activation='sigmoid'))


In [61]:
ann_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


### Model Training¶

In [62]:
ann_model.fit(x_train,y_train,batch_size=100,epochs=200,validation_data=(x_test,y_test),shuffle=True)



Epoch 1/200
46/46 [==============================] - 0s 3ms/step - loss: 0.7970 - accuracy: 0.4851 - val_loss: 0.6560 - val_accuracy: 0.5825
Epoch 2/200
46/46 [==============================] - 0s 1ms/step - loss: 0.6262 - accuracy: 0.6366 - val_loss: 0.5846 - val_accuracy: 0.6842
Epoch 3/200
46/46 [==============================] - 0s 1ms/step - loss: 0.5573 - accuracy: 0.7160 - val_loss: 0.5247 - val_accuracy: 0.7456
Epoch 4/200
46/46 [==============================] - 0s 1ms/step - loss: 0.4972 - accuracy: 0.7800 - val_loss: 0.4754 - val_accuracy: 0.7772
Epoch 5/200
46/46 [==============================] - 0s 1ms/step - loss: 0.4481 - accuracy: 0.8118 - val_loss: 0.4397 - val_accuracy: 0.8140
Epoch 6/200
46/46 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8395 - val_loss: 0.4133 - val_accuracy: 0.8307
Epoch 7/200
46/46 [==============================] - 0s 2ms/step - loss: 0.3887 - accuracy: 0.8502 - val_loss: 0.3975 - val_accuracy: 0.8368
Epoch 8/200
4

Epoch 59/200
46/46 [==============================] - 0s 1ms/step - loss: 0.2035 - accuracy: 0.9254 - val_loss: 0.2706 - val_accuracy: 0.8982
Epoch 60/200
46/46 [==============================] - 0s 1ms/step - loss: 0.2003 - accuracy: 0.9254 - val_loss: 0.2707 - val_accuracy: 0.9009
Epoch 61/200
46/46 [==============================] - 0s 1ms/step - loss: 0.2003 - accuracy: 0.9287 - val_loss: 0.2688 - val_accuracy: 0.9018
Epoch 62/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1976 - accuracy: 0.9307 - val_loss: 0.2679 - val_accuracy: 0.9000
Epoch 63/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1960 - accuracy: 0.9296 - val_loss: 0.2696 - val_accuracy: 0.9009
Epoch 64/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1941 - accuracy: 0.9281 - val_loss: 0.2704 - val_accuracy: 0.8965
Epoch 65/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1932 - accuracy: 0.9309 - val_loss: 0.2665 - val_accuracy: 0.9009
Epoch 

46/46 [==============================] - 0s 1ms/step - loss: 0.1346 - accuracy: 0.9537 - val_loss: 0.2374 - val_accuracy: 0.9175
Epoch 117/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1336 - accuracy: 0.9531 - val_loss: 0.2349 - val_accuracy: 0.9175
Epoch 118/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1336 - accuracy: 0.9522 - val_loss: 0.2475 - val_accuracy: 0.9219
Epoch 119/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1330 - accuracy: 0.9529 - val_loss: 0.2338 - val_accuracy: 0.9149
Epoch 120/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1314 - accuracy: 0.9548 - val_loss: 0.2379 - val_accuracy: 0.9228
Epoch 121/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1319 - accuracy: 0.9531 - val_loss: 0.2298 - val_accuracy: 0.9219
Epoch 122/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1300 - accuracy: 0.9546 - val_loss: 0.2383 - val_accuracy: 0.9237
Epoch 123/200

46/46 [==============================] - 0s 1ms/step - loss: 0.1021 - accuracy: 0.9629 - val_loss: 0.2426 - val_accuracy: 0.9289
Epoch 174/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1021 - accuracy: 0.9625 - val_loss: 0.2326 - val_accuracy: 0.9263
Epoch 175/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1017 - accuracy: 0.9621 - val_loss: 0.2367 - val_accuracy: 0.9281
Epoch 176/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0996 - accuracy: 0.9634 - val_loss: 0.2387 - val_accuracy: 0.9246
Epoch 177/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1001 - accuracy: 0.9614 - val_loss: 0.2543 - val_accuracy: 0.9281
Epoch 178/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0988 - accuracy: 0.9643 - val_loss: 0.2535 - val_accuracy: 0.9246
Epoch 179/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1016 - accuracy: 0.9625 - val_loss: 0.2307 - val_accuracy: 0.9228
Epoch 180/200

## convertiing probabilities to labels

In [63]:
y_pred_train = ann_model.predict(x_train)
for i in range(0,len(y_pred_train)):
    if y_pred_train[i]>=.5:
        y_pred_train[i] = 1
    else:
        y_pred_train[i] = 0

In [64]:
y_pred_test = ann_model.predict(x_test)
for i in range(0,len(y_pred_test)):
    if y_pred_test[i]>=.5:
        y_pred_test[i] = 1
    else:
        y_pred_test[i] = 0
   

In [65]:
my_dict["model"].append('ANN')
my_dict["train accuracy"].append(round(accuracy_score(y_train,y_pred_train)*100,2))
my_dict["test accuracy"].append(round(accuracy_score(y_test,y_pred_test)*100,2))
my_dict["precision"].append(round(precision_score(y_test,y_pred_test)*100,2))
my_dict["recall"].append(round(recall_score(y_test,y_pred_test)*100,2))

### Naive Bayes

In [66]:
gnb = GaussianNB()
# evaluate model
cross_validate_model(gnb)

Accuracy: 0.795 (0.018)


In [67]:
gnb.fit(x_train,y_train)
populate_matrix(gnb)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()', 'ANN', 'GaussianNB()'], 'train accuracy': [78.31, 100.0, 100.0, 94.61, 92.59, 96.93, 79.78], 'test accuracy': [76.75, 98.25, 99.91, 94.12, 89.3, 93.25, 78.95], 'precision': [75.63, 96.77, 100.0, 99.8, 89.44, 91.99, 79.26], 'recall': [78.95, 99.82, 99.82, 88.42, 89.12, 94.74, 78.42]}


### XGBoost
#### Adding performance metrics of xgboost manually, i implemented the same in seperate file

In [69]:
my_dict["model"].append('XGBClassifier()')
my_dict["train accuracy"].append(90.2)
my_dict["test accuracy"].append(90.76)
my_dict["precision"].append(93.52)
my_dict["recall"].append(87.62)

## Final performance matrix of all the models

In [72]:
pd.DataFrame(my_dict).sort_values(by='test accuracy',ascending=False)

,model,train accuracy,test accuracy,precision,recall
2,RandomForestClassifier(),100.00,99.91,100.00,99.82
1,DecisionTreeClassifier(),100.00,98.25,96.77,99.82
3,GradientBoostingClassifier(),94.61,94.12,99.80,88.42
5,ANN,96.93,93.25,91.99,94.74
7,XGBClassifier(),90.20,90.76,93.52,87.62
4,SVC(),92.59,89.30,89.44,89.12
6,GaussianNB(),79.78,78.95,79.26,78.42
0,LogisticRegression(),78.31,76.75,75.63,78.95
